In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
import codecs
import re

In [11]:
file = "sabatini-r.-odisseya-kapitana-blada-getlib.ru.txt"

In [16]:
Sequence_length = 20

New_chapter = re.compile("^\d+\.")
Unnecessary = [':', '\"', '\'', '(', ')', ',', ':', '-', '_', '/', '«', '—', '–', '»', '[', ']', '„', '“',
               '&', '#', '#']
End_symbols = ['?', ';', '!', '..', '…']
End_symbol = '.'


def encode_symbol(symbol: str, encoder: dict[str, int]) -> list[int]:
    encoded_symbol = [0] * len(encoder)
    encoded_symbol[encoder[symbol]] = 1
    return encoded_symbol


def encode_string(string: str, encoder: dict[str, int]) -> list[list[int]]:
    encoded_string = []
    for symbol in string:
        encoded_string.append(encode_symbol(symbol, encoder))
    return encoded_string


def encode_data(data: list[str]) -> tuple[list[list[list[int]]], dict[str, int], dict[int, str]]:
    symbols = list(set([symbol for string in data for symbol in string]))
    count_symbols = len(symbols)
    encoder = dict(zip(symbols, range(count_symbols)))
    encoded_data = []
    for string in tqdm(data, desc="Encode sentences"):
        encoded_data.append(encode_string(string, encoder))
    return encoded_data, encoder, {v: k for k, v in encoder.items()}


def load_data(file_name: str):
    data = []
    text = ""
    dataset_file = codecs.open(file_name, 'r', encoding='utf_8_sig')
    for line in dataset_file.readlines():
        line = line.strip()
        if (len(line) == 0) or line.isupper() or New_chapter.match(line):
            continue

        line = line.lower()
        for char in Unnecessary:
            line = line.replace(char, ' ')
        for char in End_symbols:
            line = line.replace(char, End_symbol)
        line = " ".join(line.split())
        lines = line.split(sep='.')
        for l in lines:
            l = l.strip()
            if len(l) == 0:
                continue
            data.append(l + End_symbol)
            text += l + End_symbol
    dataset_file.close()
    return data, *encode_data(data), text

In [17]:
input_sentences, encoded_sentences, encoder_map, decoder_map, input_text = load_data(file)

Encode sentences: 100%|██████████| 7768/7768 [00:01<00:00, 5112.73it/s]


In [25]:
input_sentences[0]

'рафаэль сабатини.'

In [21]:
decoder_map

{0: 'p',
 1: 'i',
 2: '4',
 3: '2',
 4: 'и',
 5: '0',
 6: 'ш',
 7: 'к',
 8: 'з',
 9: 'щ',
 10: 'й',
 11: 'r',
 12: 'v',
 13: 'о',
 14: 'м',
 15: '′',
 16: 'д',
 17: '7',
 18: 't',
 19: 'г',
 20: 'n',
 21: 'u',
 22: 'h',
 23: '3',
 24: 'т',
 25: 'a',
 26: '.',
 27: 'j',
 28: 'у',
 29: 'п',
 30: 'm',
 31: 'е',
 32: 'а',
 33: 'х',
 34: '6',
 35: 'с',
 36: '8',
 37: 'd',
 38: 'б',
 39: '9',
 40: 'o',
 41: '1',
 42: 'b',
 43: 'н',
 44: 'ь',
 45: 'ъ',
 46: 'л',
 47: '5',
 48: 'c',
 49: 'x',
 50: '́',
 51: 'ю',
 52: 'f',
 53: 'э',
 54: 'я',
 55: 'ё',
 56: 'q',
 57: 'в',
 58: 'ч',
 59: 'ы',
 60: '°',
 61: 'ц',
 62: ' ',
 63: 'e',
 64: 's',
 65: 'l',
 66: 'ж',
 67: 'ф',
 68: 'g',
 69: 'р'}

In [22]:
encoder_map

{'p': 0,
 'i': 1,
 '4': 2,
 '2': 3,
 'и': 4,
 '0': 5,
 'ш': 6,
 'к': 7,
 'з': 8,
 'щ': 9,
 'й': 10,
 'r': 11,
 'v': 12,
 'о': 13,
 'м': 14,
 '′': 15,
 'д': 16,
 '7': 17,
 't': 18,
 'г': 19,
 'n': 20,
 'u': 21,
 'h': 22,
 '3': 23,
 'т': 24,
 'a': 25,
 '.': 26,
 'j': 27,
 'у': 28,
 'п': 29,
 'm': 30,
 'е': 31,
 'а': 32,
 'х': 33,
 '6': 34,
 'с': 35,
 '8': 36,
 'd': 37,
 'б': 38,
 '9': 39,
 'o': 40,
 '1': 41,
 'b': 42,
 'н': 43,
 'ь': 44,
 'ъ': 45,
 'л': 46,
 '5': 47,
 'c': 48,
 'x': 49,
 '́': 50,
 'ю': 51,
 'f': 52,
 'э': 53,
 'я': 54,
 'ё': 55,
 'q': 56,
 'в': 57,
 'ч': 58,
 'ы': 59,
 '°': 60,
 'ц': 61,
 ' ': 62,
 'e': 63,
 's': 64,
 'l': 65,
 'ж': 66,
 'ф': 67,
 'g': 68,
 'р': 69}

In [24]:
np.array(encoded_sentences[0])

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [26]:
def get_data(sentences, length=Sequence_length):
    xs = []
    ys = []
    for sentence in tqdm(sentences, desc="Cut sentences"):
        if len(sentence) <= length:
            continue
        for i in range(0, len(sentence) - length):
            x = sentence[i:i + length]
            y = sentence[i + length]
            xs.append(x)
            ys.append(y)
    return np.array(xs), np.array(ys)

data_x, data_y = get_data(encoded_sentences)
data_x.shape, data_y.shape

Cut sentences: 100%|██████████| 7768/7768 [00:00<00:00, 14709.35it/s]


((424984, 20, 70), (424984, 70))

In [27]:
Epochs = 30

def create_lstm_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(128, input_shape=(data_x.shape[1], data_x.shape[2])),
        tf.keras.layers.Dense(data_y.shape[1], activation='softmax')]
    )
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model


def fit_lstm_model(model, x, y):
    model.fit(x, y, epochs=Epochs, verbose=1)


def sample(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_predictions = np.exp(predictions)
    predictions = exp_predictions / np.sum(exp_predictions)
    probabilities = np.random.multinomial(1, predictions, 1)
    return np.argmax(probabilities)


def predict_lstm_model(model, start_string, encoder, decoder):
    print(start_string, end="")
    start_string = encode_string(start_string, encoder)
    while True:
        prediction = model.predict(np.array([start_string]), verbose=0)
        index = sample(prediction[0], 0.5)
        result = decoder[index]
        print(result, end="")
        if result == End_symbol:
            return
        start_string.append(encode_symbol(result, encoder))
        start_string = start_string[1:]


In [28]:
lstm = create_lstm_model()
fit_lstm_model(lstm, data_x, data_y)

Epoch 1/30
13281/13281 [==============================] - 143s 11ms/step - loss: 2.3464
Epoch 2/30
13281/13281 [==============================] - 167s 13ms/step - loss: 1.9725
Epoch 3/30
13281/13281 [==============================] - 139s 10ms/step - loss: 1.8156
Epoch 4/30
13281/13281 [==============================] - 146s 11ms/step - loss: 1.7192
Epoch 5/30
13281/13281 [==============================] - 156s 12ms/step - loss: 1.6534
Epoch 6/30
13281/13281 [==============================] - 149s 11ms/step - loss: 1.6062
Epoch 7/30
13281/13281 [==============================] - 153s 12ms/step - loss: 1.5698
Epoch 8/30
13281/13281 [==============================] - 156s 12ms/step - loss: 1.5407
Epoch 9/30
13281/13281 [==============================] - 154s 12ms/step - loss: 1.5175
Epoch 10/30
13281/13281 [==============================] - 155s 12ms/step - loss: 1.4978
Epoch 11/30
13281/13281 [==============================] - 169s 13ms/step - loss: 1.4812
Epoch 12/30
13281/13281 [=====

In [34]:
start = "капитан блад поднял"
predict_lstm_model(lstm, start, encoder_map, decoder_map)

капитан блад поднял его с долго на который вновь проблизнился к нему из голландся губернатора появилась и всё же не можете оказаться он был головой ему приказал по себе на свою капитан блад и в бледном кораблей и вернулся и ответила и что объяснялись взять этого лица вы не насмешки и не заметил блад и просто никакого придум на него сказал собраться с ним и прибыл к капитан блад прибыли находился в сторону обосяк принял их по испански и добавил на рассказал его вы намереваясь на него с подавающих насторилась на него отправляет с них привести потому что был под того как до того как был только до того что он не задержался в себя и быть с тем погладела в барбадос а в таком судеать своего как не приняла не сказать ваш не была повесить и это сильными голландских море до того своего подвергаться к нему стали одно и добраться до корабль который был на борту и назначилась от бросили раздражали пристально заметно в которых он не раздражённое высокий ваших полученными на службу к берегу с заключа

In [57]:
Suffix_length = 4

def create_markov_model(text):
    counts = dict()
    for i in tqdm(range(0, len(text) - Suffix_length - 1), desc="Read sentences"):
        cur_symbols = text[i: i + Suffix_length]
        next_symbols = text[i + 1: i + Suffix_length + 1]
        if cur_symbols not in counts:
            counts[cur_symbols] = dict()
        if next_symbols not in counts[cur_symbols]:
            counts[cur_symbols][next_symbols] = 0
        counts[cur_symbols][next_symbols] += 1
    probabilities = dict()
    for cur, dict_next in tqdm(counts.items(), desc="Count probabilities"):
        next_to_int = dict()
        next_probabilities = []
        next_number = 0
        counts_sum = sum(dict_next.values())
        for next_symbols, count in dict_next.items():
            next_to_int[next_symbols] = next_number
            next_number += 1
            next_probabilities.append(count / counts_sum)
        probabilities[cur] = (next_probabilities, next_to_int, {v: k for k, v in next_to_int.items()})
    return probabilities


def predict_markov(model, string):
    if string not in model:
        print('No such text in Markov model')
        return '.'

    next_symbols, next_to_int, int_to_next = model[string]
    index = sample(next_symbols, 0.5)
    return int_to_next[index][-1]


def predict_markov_model(model, start_string):
    print(start_string, end="")
    start_string = start_string[-Suffix_length:]
    while True:
        prediction = predict_markov(model, start_string)
        if prediction is None:
            return
        print(prediction, end="")
        if prediction == End_symbol:
            return
        start_string += prediction
        start_string = start_string[1:]

In [58]:
markov_model = create_markov_model(input_text)

Count probabilities: 100%|██████████| 38959/38959 [00:00<00:00, 371981.49it/s]


In [59]:
predict_markov_model(markov_model, 'тут л')

тут лорд джулиан сказал он сказал он под после этого подошёл вслед за чтобы находит что уважением обратил внимать мои людей по том что вашей случай мне как со мне как он соответил блада сказал он.